<a href="https://colab.research.google.com/github/jameramellyn/skincare_project/blob/main/K_cross_SkinTypeDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
!pip install Pillow
#train test split
from sklearn.model_selection import train_test_split

from numpy import asarray

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from sklearn.metrics import confusion_matrix


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd
images = '/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types'

/content


In [4]:
#the dataset only comes with images that have been sorted into folders, but not formally labelled
#this function will help to label each of the images as the dataframe is being created
label_index = {"dry": 0, "normal": 1, "oily": 2}
index_label = {0: "dry", 1: "normal", 2: "oily"}

def create_df(base):
    dd = {"images": [], "labels": []}
    for i in os.listdir(base):
        label = os.path.join(base, i)
        for j in os.listdir(label):
            img = os.path.join(label, j)
            dd["images"] += [img]
            dd["labels"] += [label_index[i]]
    return pd.DataFrame(dd)


train_df = create_df("/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/train")
val_df = create_df("/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/valid")
test_df = create_df("/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/test")

df = pd.concat([train_df, val_df, test_df], axis=0)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/train'

In [ ]:
def load_image(file_path):
    with Image.open(file_path) as img:
        img = img.convert('RGB')  # Ensure image is in RGB mode
        return np.array(img)

def extract_image_data(file_path):
    img_array = load_image(file_path)
    height, width, channels = img_array.shape
    return pd.Series({
        'height': height,
        'width': width,
        'channels': channels,
        'image_array': img_array
    })

# Apply the function to each file path in the dataframe
df_data = train_df.iloc[:, 0].apply(extract_image_data)


In [ ]:
df_data.head()

In [ ]:
df = pd.concat([df_data, train_df.iloc[:, 1]], axis=1)
df.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
import matplotlib.pyplot as plt

def plotLoss(history, fold_number):
    TRAINING_LOSS = history.history['loss']
    VALIDATION_LOSS = history.history.get('val_loss')  # Validation loss if available
    EPOCHS = range(1, len(TRAINING_LOSS) + 1)

    plt.figure(figsize=(8, 6))

    # Plot training loss
    plt.plot(EPOCHS, TRAINING_LOSS, 'b', label='Training Loss')

    # Plot validation loss (if available)
    if VALIDATION_LOSS:
        plt.plot(EPOCHS, VALIDATION_LOSS, 'r', label='Validation Loss')

    # Set y-axis to range from 0 to 5
    plt.ylim(0, 5)

    plt.title(f'Training and Validation Loss (Fold {fold_number})')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.show()


In [ ]:
import matplotlib.pyplot as plt

def plotAccuracy(history, fold_number):
    TRAINING_ACCURACY = history.history['accuracy']
    VALIDATION_ACCURACY = history.history.get('val_accuracy')  # Validation accuracy if available
    EPOCHS = range(1, len(TRAINING_ACCURACY) + 1)

    plt.figure(figsize=(8, 6))

    # Plot training accuracy
    plt.plot(EPOCHS, TRAINING_ACCURACY, 'b', label='Training Accuracy')

    # Plot validation accuracy (if available)
    if VALIDATION_ACCURACY:
        plt.plot(EPOCHS, VALIDATION_ACCURACY, 'r', label='Validation Accuracy')

    # Set y-axis to range from 0 to 1 (as accuracy is typically between 0 and 1)
    plt.ylim(0, 1)

    plt.title(f'Training and Validation Accuracy (Fold {fold_number})')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.show()


In [ ]:
#defining the variables
height = 640
width = 640
num_classes = 3
channels = 3

# normalize
def normalize(df):
    normalized_images = []
    for index, row in df.iterrows():
      # Extract the image array from the first column
      image_array = row[3]
      # Ensure the array is in float32 format
      image_array = image_array.astype('float32')
      # Normalize the pixel values to the range [0, 1]
      image_array /= 255.0
      # Append the normalized image to a new list
      normalized_images.append(image_array)
    # Create a new DataFrame with the normalized images
    df[0] = normalized_images
    return df

def kCross(df,i):
    # Split the data into training and testing sets
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=i)
    x_train, y_train = train_df.drop('labels', axis=1), train_df['labels']
    x_test, y_test = test_df.drop('labels', axis=1), test_df['labels']

    #Normalize training and testing data
    x_train_norm = normalize(x_train)
    x_test_norm = normalize(x_test)

    # Extract the normalized image arrays and convert them to a NumPy array
    x_train = np.stack(x_train_norm['image_array'].values)
    x_test = np.stack(x_test_norm['image_array'].values)

    #validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=i)

    #transferring to one-hot
    y_train_oh = tf.keras.utils.to_categorical(y_train, 3)
    y_test_oh = tf.keras.utils.to_categorical(y_test, 3)
    y_val_oh = tf.keras.utils.to_categorical(y_val, 3)

    # create CNN structure
    model = Sequential()
    model.add(Input(shape=(height, width, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu')) # notice no MaxPool after this one
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.summary()

    # compile and train model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model with validation data and capture the history
    history = model.fit(x_train, y_train_oh, epochs=10, batch_size=32, validation_data=(x_val, y_val_oh))

    # Plot the training/validation loss for this fold
    plotLoss(history, fold_number=i+1)
    plotAccuracy(history, fold_number=i+1)

    # evaluate accuracy
    y_pred = model.predict(x_test)
    y_pred = tf.argmax(y_pred, axis=1)
    loss, acc = model.evaluate(x_test, y_test_oh)

    print(f"Fold {i+1} - Test accuracy:", acc)
    print(f"Confusion matrix for fold {i+1}:\n", confusion_matrix(y_test, y_pred))



In [ ]:
for i in range(5):
  kCross(df,i)